In [327]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix



In [328]:
excel_file = 'test_data.xlsm'
test_data = pd.read_excel(excel_file)
df = test_data


In [329]:
df.head(20)

,YEAR,MONTH,DAY_OF_MONTH,ORIGIN_CITY_NAME,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP
0,2018,1,27,"Fort Lauderdale, FL","Houston, TX",615,602,-13,19,621,749,7,808,756,-12,173,174,966,4
1,2018,1,27,"Seattle, WA","San Francisco, CA",618,614,-4,16,630,808,5,831,813,-18,133,119,679,3
2,2018,1,27,"Washington, DC","Houston, TX",830,828,-2,17,845,1055,13,1107,1108,1,217,220,1208,5
3,2018,1,27,"Los Angeles, CA","Chicago, IL",650,641,-9,17,658,1230,12,1250,1242,-8,240,241,1744,7
4,2018,1,27,"Jacksonville, FL","Newark, NJ",1824,1810,-14,11,1821,2013,8,2045,2021,-24,141,131,820,4
5,2018,1,27,"Houston, TX","Phoenix, AZ",1420,1413,-7,19,1432,1554,5,1618,1559,-19,178,166,1009,5
6,2018,1,27,"Newark, NJ","Honolulu, HI",815,842,27,21,903,1454,3,1438,1457,19,683,675,4962,11
7,2018,1,27,"Honolulu, HI","Newark, NJ",1615,1623,8,21,1644,618,12,653,630,-23,578,547,4962,11
8,2018,1,27,"Las Vegas, NV","San Francisco, CA",1520,1515,-5,11,1526,1634,7,1703,1641,-22,103,86,414,2
9,2018,1,27,"Washington, DC","Tampa, FL",1753,1746,-7,16,1802,1955,6,2019,2001,-18,146,135,811,4


In [330]:
df = df.drop(['YEAR','MONTH','DAY_OF_MONTH','ORIGIN_CITY_NAME','DEST_CITY_NAME','TAXI_OUT','WHEELS_OFF','WHEELS_ON','DISTANCE','DISTANCE_GROUP','TAXI_IN','CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME','ARR_DELAY','DEP_DELAY'], axis=1)


In [331]:
df.head(8)

,CRS_DEP_TIME,DEP_TIME,CRS_ARR_TIME,ARR_TIME
0,615,602,808,756
1,618,614,831,813
2,830,828,1107,1108
3,650,641,1250,1242
4,1824,1810,2045,2021
5,1420,1413,1618,1559
6,815,842,1438,1457
7,1615,1623,653,630


In [332]:
#add another column
df['outcome'] = df['ARR_TIME'] - df['CRS_ARR_TIME']

In [351]:
#delayed column
#if flights are delayed more than 15minutes it will show a 1
df['delayed'] = np.where(df['outcome']>=15, 1, 0)

In [352]:
data = df[df.columns[:-1]]
label = df["delayed"]

In [358]:
df.head(80)

,CRS_DEP_TIME,DEP_TIME,CRS_ARR_TIME,ARR_TIME,outcome,delayed
0,615,602,808,756,-52,0
1,618,614,831,813,-18,0
2,830,828,1107,1108,1,0
3,650,641,1250,1242,-8,0
4,1824,1810,2045,2021,-24,0
5,1420,1413,1618,1559,-59,0
6,815,842,1438,1457,19,1
7,1615,1623,653,630,-23,0
8,1520,1515,1703,1641,-62,0
9,1753,1746,2019,2001,-18,0


In [359]:
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

Train size: 240
Test size: 60


In [360]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X=df[['CRS_ARR_TIME', 'ARR_TIME']]  # Features
y=df['delayed']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [361]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=20,random_state=0)  


#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [362]:
#accuracy calculation
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9555555555555556
